In [5]:
import os
import sys
sys.path.append(os.path.realpath('../..'))
import aoc
my_aoc = aoc.AdventOfCode(2022,19)

In [6]:
input_text = """Blueprint 1:
  Each ore robot costs 4 ore.
  Each clay robot costs 2 ore.
  Each obsidian robot costs 3 ore and 14 clay.
  Each geode robot costs 2 ore and 7 obsidian.

Blueprint 2:
  Each ore robot costs 2 ore.
  Each clay robot costs 3 ore.
  Each obsidian robot costs 3 ore and 8 clay.
  Each geode robot costs 3 ore and 12 obsidian."""

In [7]:
"""
Notes:
    I'm thinking recursion since we have a limited time span of 24 minutes, so recursion depth won't be a problem.
    This would prevent the memory consumption of a large heap.

    Also convert the blueprint to json for passing, and use an lru_cached function to decode the json back to dict.



"""
import json
from functools import lru_cache

@lru_cache(maxsize=None)
def decode_blueprint(json_text):
    """Cached function for decoding json blueprint"""
    return json.loads(json_text)

def parse_data(text):
    blueprints = {}
    blueprint_text_list = text.split('\n\n')
    for blueprint_text in blueprint_text_list:
        lines = blueprint_text.splitlines()
        blueprint_id = int(lines.pop(0).split(' ')[1].replace(':',''))
        blueprints[blueprint_id] = {}
        while lines:
            line = lines.pop(0)
            robot, materials = line.split(' costs ')
            robot = robot.split(' ')[3]
            blueprints[blueprint_id][robot] = {}
            materials = materials.replace('.','')
            for material in materials.split(' and '):
                count, element = material.split(' ')
                count = int(count)
                blueprints[blueprint_id][robot][element] = count
    return blueprints

element_map = {
    "ore": 0,
    "clay": 1,
    "obsidian": 2,
    "geode": 3
}

@lru_cache(maxsize=None)
def can_build(inventory, materials):
    materials = decode_blueprint(materials)
    for element, count in materials.items():
        if inventory[element_map[element]] < count:
            return False
    return True
    
def build(minute, inventory, robots, blueprint_json, robot):
    # print(f"build({minute}, {inventory}, {robots}, blueprint_json, {robot})")
    if sum(robots) > 25:
        print(f"too many robots, this isn't possible")
        raise ValueError
    blueprint = decode_blueprint(blueprint_json)
    inventory_list = list(inventory)
    robots_list = list(robots)
    materials = blueprint[robot]
    # subtract materials from inventory
    for element, count in materials.items():
        inventory_list[element_map[element]] -= count
    # mine with existing robots:
    for idx, count in enumerate(robots):
        inventory_list[idx] += count
    # add new robot to robots
    # print(f"before robot: {robot}, element_map[robot]: {element_map[robot]}, robots_list[element_map[robot]]: {robots_list[element_map[robot]]}")
    robots_list[element_map[robot]] += 1
    # print(f"after robot: {robot}, element_map[robot]: {element_map[robot]}, robots_list[element_map[robot]]: {robots_list[element_map[robot]]}")
    return do_turn(minute +1, tuple(inventory_list), tuple(robots_list), blueprint_json)

def mine(minute, inventory, robots, blueprint_json):
    # print(f"mine({minute}, {inventory}, {robots}, blueprint_json)")
    inventory_list = list(inventory)
    for idx, count in enumerate(robots):
        inventory_list[idx] += count
    return do_turn(minute + 1, tuple(inventory_list), robots, blueprint_json)

@lru_cache(maxsize=None)
def compare(new_inventory, inventory):
    # print(f"compare({new_inventory}, {inventory})")
    for element in range(len(inventory) - 1, -1, -1):
        # print(element)
        if new_inventory[element] == inventory[element]:
            continue
        return  new_inventory[element] > inventory[element]
    # they are identical
    return False 

# seen = set()

# counters = {"RecursionError": 0}

# @lru_cache(maxsize=None)
def do_turn(minute, inventory, robots, blueprint_json):
    # print(f"do_turn({minute}, {inventory}, {robots}, blueprint_json)")
    # if (minute, inventory, robots, blueprint_json) in seen:
    #     print(f"state already visited: {(minute, inventory, robots, blueprint_json)}")
    #     counters['RecursionError'] += 1
    #     if counters['RecursionError'] > 100:
    #         raise RecursionError
    
    now = time.time()
    if now - start_time > 300:
        print("Time limit reached")
        raise TimeoutError
    # seen.add((minute, inventory, robots, blueprint_json))
    blueprint = decode_blueprint(blueprint_json)
    if minute > 24:
        return inventory, robots
    # get state if we mine but don't build
    mineonly_inventory, mineonly_robots = inventory, robots
    # get state for building each robot type we can build
    build_inventory = inventory
    build_robots = robots
    builds = 0
    for robot, materials in blueprint.items():
        if can_build(inventory, json.dumps(materials)):
            new_robots, new_inventory = build(minute, inventory, robots, blueprint_json, robot)
            builds +=1 
            if compare(new_inventory, build_inventory):
                build_robots = new_robots
                build_inventory = new_inventory
    mineonly_inventory, mineonly_robots = mine(minute, inventory, robots, blueprint_json)
    if compare(build_inventory, inventory):
        inventory = build_inventory
        robot = build_robots   
    if compare(mineonly_inventory, inventory):
        inventory = mineonly_inventory
        robot = mineonly_robots    
    # print("do_turn: ", do_turn.cache_info())
    # print("compare: ", compare.cache_info())
    # print("can_build: ", can_build.cache_info())
    # print("decode_blueprint: ", decode_blueprint.cache_info())
    
    return inventory, robots


In [8]:
import time
start_time = time.time()
blueprints = parse_data(input_text)
quality_levels = {}
for blueprint_id, blueprint in blueprints.items():
    print(blueprint_id, blueprint)
    quality_levels[blueprint_id] = do_turn(1, (0, 0, 0, 0), (1, 0, 0, 0), json.dumps(blueprint))
    print(quality_levels)



1 {'ore': {'ore': 4}, 'clay': {'ore': 2}, 'obsidian': {'ore': 3, 'clay': 14}, 'geode': {'ore': 2, 'obsidian': 7}}
Time limit reached


TimeoutError: 

In [ ]:
test = frozenset(blueprints)
print(test)